<a href="https://colab.research.google.com/github/Nandan-2004/DetectNCount/blob/main/DetectNCount.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 1️⃣ Install and Import
!pip install -q ultralytics
from ultralytics import YOLO

import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from google.colab import files
from IPython.display import HTML, display
import base64
from collections import Counter
import pandas as pd

# Load YOLOv8 model (you can switch to 'yolov8m.pt' or 'yolov8x.pt' for better accuracy)
model = YOLO('yolov8x.pt')

# 2️⃣ Upload files
uploaded = files.upload()  # upload one or more images/videos

# 3️⃣ Process Each File
for fname in uploaded.keys():
    ext = os.path.splitext(fname)[1].lower()

    if ext in ['.jpg', '.jpeg', '.png', '.jfif']:
        # ----- IMAGE PROCESSING -----
        results = model(fname)[0]
        annotated = results.plot()  # Annotated image (BGR)

        # Per-class counting
        class_ids = results.boxes.cls.cpu().numpy().astype(int)
        class_names = [model.names[i] for i in class_ids]
        class_counts = Counter(class_names)

        # Prepare display text
        summary_text = "\n".join([f"{cls}: {cnt}" for cls, cnt in class_counts.items()])


        # Display with matplotlib
        plt.figure(figsize=(10, 8))
        plt.imshow(cv2.cvtColor(annotated, cv2.COLOR_BGR2RGB))
        plt.title(f"{fname} — Total Detected: {len(results.boxes)}")
        plt.axis('off')
       # plt.figtext(0.5, 0.01, summary_text, wrap=True, horizontalalignment='center', fontsize=12)
        plt.show()

        # Optional: save annotated image
        cv2.imwrite(f"annotated_{fname}", annotated)

    elif ext in ['.mp4', '.avi', '.mov', '.mkv']:
        # ----- VIDEO PROCESSING -----
        cap = cv2.VideoCapture(fname)
        fps = cap.get(cv2.CAP_PROP_FPS) or 25
        w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

        out_name = f"annotated_{fname}"
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        writer = cv2.VideoWriter(out_name, fourcc, fps, (w, h))

        while True:
            ret, frame = cap.read()
            if not ret:
                break

            res = model(frame)[0]
            annotated = res.plot()

            # Count per class
            class_ids = res.boxes.cls.cpu().numpy().astype(int)
            class_names = [model.names[i] for i in class_ids]
            class_counts = Counter(class_names)

            # Overlay count info on frame
            y_offset = 50
            for cls_name, count in class_counts.items():
                text = f"{cls_name}: {count}"
                cv2.putText(annotated, text, (10, y_offset), cv2.FONT_HERSHEY_SIMPLEX,
                            0.7, (0, 255, 0), 2)
                y_offset += 25

            writer.write(annotated)

        cap.release()
        writer.release()
        from google.colab import files
        files.download(out_name)  # e.g., "annotated_video.mp4"
    else:
        print(f"Unsupported file type: {fname}")
results = model(fname, verbose=False)[0]
# Get all detections from the result
boxes = results.boxes

# Total number of objects detected
object_count = len(boxes)
print("Total objects detected:", object_count)

# Get class IDs and count them
class_ids = boxes.cls.cpu().numpy().astype(int)
counts = Counter(class_ids)

# Print count per class using model.names
for cls_id, count in counts.items():
    class_name = model.names[cls_id]
    print(f"{class_name}: {count}")
